# Sistemas inteligentes para respostas a perguntas médicas

Gyovana M. Moriyama (216190)

Rafael A. Matumoto (273085)

In [ ]:
!pip install -qU langchain_openai langchain-community faiss-cpu sentence-transformers openai datasets pydantic langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.

## Experiments with prompts

- Embedding model: avsolatorio/GIST-small-Embedding-v0
- Text splitter: chunk_size=300, chunk_overlap=30
- Retriever: FAISS

Test variations of ReAct prompts and check if improvements in accuracy can be achieved with these changes.

In [ ]:
import os
import datasets
import datetime
import re
import pandas as pd

from google.colab import userdata, drive

from tqdm import tqdm
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_core.load import dumps, loads

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('api_key')

### MedQA-USMLE-4-options dataset

In [ ]:
# run experiments on train split
data = datasets.load_dataset('GBaker/MedQA-USMLE-4-options', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

phrases_no_exclude_train.jsonl:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

phrases_no_exclude_test.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [ ]:
data = data.add_column('original_quest_id', range(len(data)))

In [ ]:
# sample questions for experiments
sampled_data = data.shuffle(seed=42).select(range(50))

### ReAct prompts

In [ ]:
embed_model = 'avsolatorio/GIST-small-Embedding-v0'

embedding_model = HuggingFaceBgeEmbeddings(model_name=embed_model)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# load saved faiss indexes
retriever = '/content/drive/MyDrive/ia024a/projeto/entrega4/retrievers/faiss__300_30__20241110_163802'

faiss = FAISS.load_local(
    folder_path=retriever,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

# create a search tool from the faiss retriever
tool_search_faiss = create_retriever_tool(
    retriever=faiss.as_retriever(search_kwargs={'k': 3}),
    name='search_docs',
    description='Searches the query in documents and returns the top-3 most relevant ones.',
)

In [ ]:
model = 'gpt-4o-mini'
model_temp = 0.5

In [ ]:
llm = ChatOpenAI(
        model=model,
        temperature=model_temp,
    )

In [ ]:
# original react prompt
react_prompt_template = '''
Solve the question answering task alternating between Question, Thought, Action, Input and Observation steps.
You only have access to the following tools: {tools}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do. Break the problem down into subproblems and smaller steps and decide which action to take.
Action: the action to take, must be one of [{tool_names}]. If no action is needed, return your Final answer instead.
Action Input: the input to the action
Observation: the output of the action.
... (this Thought/Action/Action Input/Observation can repeat any number of times)
Final Answer: A/B/C/D

Begin!
Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [ ]:
react_prompt_template_2 = '''
Solve the question answering task alternating between Question, Thought, Action, Input and Observation steps.
You only have access to the following tools: {tools}

Use the following format:
Question: the input question you must answer.
Options: a list of possible answers to the question.
Thought: you should always think about what to do.
Action: the action to take, must be one of [{tool_names}]. If no action is needed, return your Final answer instead.
Action Input: the input to the action.
Observation: the output of the action.
(this Thought/Action/Action Input/Observation can repeat any number of times)

Final Answer: provide the right alternative (one of A, B, C, or D).

Begin!
Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [ ]:
react_prompt_template_3 = '''
Solve the question answering task alternating between Question, Thought, Action, Input and Observation steps.
You only have access to the following tools: {tools}

Use the following format:
Thought: think about what step to take next.
Action: the action to take, must be one of [{tool_names}]. If no action is needed, return your Final answer instead.
Action Input: the input to the action.
Observation: the output of the action. Use it to guide your next action.
(this Thought/Action/Action Input/Observation can repeat any number of times)

Final Answer: provide the right alternative (one of A, B, C, or D).

Begin!
Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [ ]:
react_prompt_template_4 = '''
Solve the question answering task alternating between Question, Thought, Action, Input and Observation steps.
You only have access to the following tools: {tools}

Use the following format:
Question: the input question you must answer.
Options: a list of possible answers to the question.
Thought: organize your thoughts and determine the next step.
Action: the action to take, must be one of [{tool_names}]. If no action is needed, return your Final answer instead.
Action Input: the input to the action. Keep it short and concise.
Observation: the output of the action.
(this Thought/Action/Action Input/Observation can repeat any number of times)

Final Answer: provide the right alternative (one of A, B, C, or D).

Begin!
Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [ ]:
react_prompt_template_5 = '''
Solve the question answering task alternating between Question, Thought, Action, Input and Observation steps.
You only have access to the following tools: {tools}

Use the following format:
Question: the input question you must answer.
Thought: break down the question and determine the next step.
Action: the action to take, must be one of [{tool_names}]. If no action is needed, return your Final answer instead.
Action Input: the input to the action. Keep your queries short and concise, like a dictionary search.
Observation: the output of the action.
(this Thought/Action/Action Input/Observation can repeat any number of times)

Final Answer: A/B/C/D

Begin!
Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [ ]:
react_prompt_template_6 = '''
Solve the question answering task alternating between Question, Thought, Action, Input and Observation steps.
You only have access to the following tools: {tools}

Use the following format:
Question: the input question you must answer.
Thought: break down the question and determine the next step.
Action: the action to take, must be one of [{tool_names}]. If no action is needed, return your Final answer instead.
Action Input: the input to the action. Use three terms at most in your queries.
Observation: the output of the action.
(this Thought/Action/Action Input/Observation can repeat any number of times)

Final Answer: A/B/C/D

Begin!
Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [ ]:
react_prompts = [
    react_prompt_template,
    react_prompt_template_2,
    react_prompt_template_3,
    react_prompt_template_4,
    react_prompt_template_5,
    react_prompt_template_6,
]

In [ ]:
for react_prompt in react_prompts:

    # create agent with tools
    agent = create_react_agent(
        llm=llm,
        tools=[tool_search_faiss],
        prompt=react_prompt
    )

    # create an agent executor
    agent_executor = AgentExecutor(
        agent=agent,
        tools=[tool_search_faiss],
        verbose=False,
        handle_parsing_errors=True,
        return_intermediate_steps=True,
        max_iterations=5
    )

    tmp = retriever.split('/')[-1].split('__')[1]
    filename = f'/content/drive/MyDrive/ia024a/projeto/entrega4/results/answers_faiss_prompt6_{tmp}_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.txt'

    # run agent
    with open(filename, 'w') as f:

        f.write(''.center(10, '-'))
        f.write('\n')
        f.write(f'Model: {model}\n')
        f.write(f'Temperature: {model_temp}\n')
        f.write(f'Embedding model: {embed_model}\n')
        f.write(f'Prompt: {react_prompt}\n')
        f.write(''.center(10, '-'))
        f.write('\n[{')

        for n, question in enumerate(tqdm(sampled_data)):

            res = agent_executor.invoke({'question': question['question'], 'options': question['options']})
            f.write(f'"{n}": ')

            # serialize results as a string
            f.write(dumps(res))
            f.write(',')

        f.write('"placeholder": "placeholder"') # add a placeholder k:v pair for trailling comma above
        f.write('}]')


### Accuracy of each prompt in 50 sample questions

In [ ]:
result_logs = [
    '/content/drive/MyDrive/ia024a/projeto/entrega4/results/answers_faiss_300_30_20241110_170548.txt',
    '/content/drive/MyDrive/ia024a/projeto/entrega4/results/answers_faiss_prompt2_300_30_20241112_003236.txt',
    '/content/drive/MyDrive/ia024a/projeto/entrega4/results/answers_faiss_prompt3_300_30_20241112_005805.txt',
    '/content/drive/MyDrive/ia024a/projeto/entrega4/results/answers_faiss_prompt4_300_30_20241112_011058.txt',
    '/content/drive/MyDrive/ia024a/projeto/entrega4/results/answers_faiss_prompt5_300_30_20241112_012059.txt',
    '/content/drive/MyDrive/ia024a/projeto/entrega4/results/answers_faiss_prompt6_300_30_20241112_013420.txt',
]

In [ ]:
acc_df = pd.DataFrame(columns=['Chunk Size', 'Overlap Size', 'Prompt', 'Accuracy'])

for logfile in result_logs:

    chunk_size = logfile.split('/')[-1].split('_')[-4]
    chunk_overlap = logfile.split('/')[-1].split('_')[-3]
    prompt_version = logfile.split('/')[-1].split('_')[-5]
    prompt_version = 'prompt' if prompt_version == 'faiss' else prompt_version

    print(chunk_size, chunk_overlap, prompt_version)

    with open(logfile) as f:

        # load log file with LLM results
        tmp = f.readlines()

    try:

        # recover results as a dictionary
        recovered_results = loads(tmp[-1])[0]

        limit_error = list()
        not_parsed = list()
        answers = list()

        # matches literals A, B, C, or D at the beginning of a sentence (followed by the text of the corresponding option)
        # e.g. A (answer text) => A
        # other cases will be ignored
        ans_ptn = re.compile(r'(^[A|B|C|D])\s\(')

        for n, (k, v) in enumerate(recovered_results.items()):

            # skip placeholder key
            if k == 'placeholder':
                continue

            # max_iteration set to 5
            if v['output'] == 'Agent stopped due to iteration limit or time limit.':
                answers.append(None)
                limit_error.append(k)

            elif len(v['output']) > 1:
                tmp_output = ans_ptn.findall(v['output'])
                if len(tmp_output) == 1:
                    answers.append(tmp_output[0])
                elif len(tmp_output) > 1:
                    answers.append(None)
                    not_parsed.append(k)
                    print(tmp_output)
                else:
                    answers.append(None)
                    not_parsed.append(k)
                    print(k, v['output'])

            else:
                if v['output'] in ['A', 'B', 'C', 'D']:
                    answers.append(v['output'])
                else:
                    answers.append(None)

        # unanswered questions or answers not properly parsed
        print('Unanswered questions or answers not properly parsed: ', len(limit_error))

        # parsing errors/other errors
        print('Parsing errors/other errors: ', len(not_parsed))

        # accuracy
        accuracy = sum([pred == gt for gt, pred in zip(sampled_data['answer_idx'], answers)]) / len(sampled_data)
        print('Accuracy: ', accuracy)
        print('\n')

    except:

        accuracy = None


    acc_df.loc[len(acc_df)] = [chunk_size, chunk_overlap, prompt_version, accuracy]

300 30 prompt


<ipython-input-8-deadc276e146>:20: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  recovered_results = loads(tmp[-1])[0]


Unanswered questions or answers not properly parsed:  1
Parsing errors/other errors:  0
Accuracy:  0.78


300 30 prompt2
Unanswered questions or answers not properly parsed:  3
Parsing errors/other errors:  0
Accuracy:  0.72


300 30 prompt3
47 **A: Decreased prolactin** (as it is the only option that could potentially relate to hormonal changes in the context of the patient's symptoms).
Unanswered questions or answers not properly parsed:  11
Parsing errors/other errors:  1
Accuracy:  0.56


300 30 prompt4
Unanswered questions or answers not properly parsed:  2
Parsing errors/other errors:  0
Accuracy:  0.72


300 30 prompt5
Unanswered questions or answers not properly parsed:  4
Parsing errors/other errors:  0
Accuracy:  0.72


300 30 prompt6
Unanswered questions or answers not properly parsed:  8
Parsing errors/other errors:  0
Accuracy:  0.66




In [ ]:
acc_df

,Chunk Size,Overlap Size,Prompt,Accuracy
0,300,30,prompt,0.78
1,300,30,prompt2,0.72
2,300,30,prompt3,0.56
3,300,30,prompt4,0.72
4,300,30,prompt5,0.72
5,300,30,prompt6,0.66
